<div style="background-color:#FCE205; padding:10px; border-radius:5px; color:black; font-weight:bold;">
    <h3>Processing Weather Data to Quarterly Intervals</h3>
</div>

The weather data collected in notebook 1a is daily data which has to be condensed to quarterly data to align the Bee dataset.

In [2]:
# Import libraries
import pandas as pd
import os

In [3]:
# set working directory
ITM_DIR = os.path.join(os.getcwd(), '../data/import')

In [4]:
# read in weather data and combine
file_names = [f'Hourly_Weather_Data_part{i}.csv' for i in range(1, 7)]
weather_data = pd.concat([pd.read_csv(os.path.join(ITM_DIR, file)) for file in file_names], ignore_index=True)
weather_data.shape

(146150, 12)

In [5]:
# confirm date is indeed daily
weather_data['date']

0         2015-01-01 00:00:00+00:00
1         2015-01-02 00:00:00+00:00
2         2015-01-03 00:00:00+00:00
3         2015-01-04 00:00:00+00:00
4         2015-01-05 00:00:00+00:00
                    ...            
146145    2022-12-28 00:00:00+00:00
146146    2022-12-29 00:00:00+00:00
146147    2022-12-30 00:00:00+00:00
146148    2022-12-31 00:00:00+00:00
146149    2023-01-01 00:00:00+00:00
Name: date, Length: 146150, dtype: object

In [6]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146150 entries, 0 to 146149
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date                       146150 non-null  object 
 1   latitude                   146150 non-null  float64
 2   longitude                  146150 non-null  float64
 3   wind_speed_10m_max         146150 non-null  float64
 4   weather_code               146150 non-null  float64
 5   temperature_2m_mean        146150 non-null  float64
 6   temperature_2m_max         146150 non-null  float64
 7   temperature_2m_min         146150 non-null  float64
 8   precipitation_hours        146150 non-null  float64
 9   relative_humidity_2m_mean  146150 non-null  float64
 10  relative_humidity_2m_max   146150 non-null  float64
 11  relative_humidity_2m_min   146150 non-null  float64
dtypes: float64(11), object(1)
memory usage: 13.4+ MB


<div style="background-color:#FCE205; padding:10px; border-radius:5px; color:black; font-weight:bold;">
    <h3>Condense weather data to represent Quarterly information</h3>
    </div>

- For weather code, the sum of how often the code appear across the quarter is calculated.

    (eg. weather_code_33 = 5 would mean that that particular code was the most severe weather code for 5 days across the quarter)

- For the daily mean temperature, both the mean and sum is taken across all days of the quarter and assigned meanmean or meansum respectively.

- For the daily mean relative humidity, both the mean and sum is taken across all days of the quarter and assigned meanmean or meansum respectively.

In [7]:
weather_data['weather_code'] = weather_data['weather_code'].astype(int)

In [8]:
# Turn weather_code into separate columns per code with 1 or 0
unique_weather_codes = weather_data['weather_code'].unique()
for code in unique_weather_codes:
    weather_data[f'weather_code_{code}'] = (weather_data['weather_code'] == code).astype(int)

In [9]:
# Condense data into a quarterly summary per latitude/longitude combination
weather_data['quarter'] = pd.to_datetime(weather_data['date']).dt.to_period('Q')

# Group by latitude, longitude, and quarter
quarterly_summary = weather_data.groupby(['latitude', 'longitude', 'quarter']).agg(
    {   # Aggregate mean columns with mean
        'temperature_2m_mean': ['mean', 'sum'],
        'relative_humidity_2m_mean': ['mean', 'sum'],
        # Aggregate max columns with max
        'wind_speed_10m_max': 'max',
        'temperature_2m_max': 'max',
        'relative_humidity_2m_max': 'max',
        # Aggregate min columns with min
        'temperature_2m_min': 'min',
        'relative_humidity_2m_min': 'min',
        # Sum counts of weather codes
        'precipitation_hours': 'sum',
        'weather_code_53': 'sum',
        'weather_code_63': 'sum',
        'weather_code_61': 'sum',
        'weather_code_65': 'sum',
        'weather_code_3': 'sum',
        'weather_code_2': 'sum',
        'weather_code_0': 'sum',
        'weather_code_51': 'sum',
        'weather_code_1': 'sum',
        'weather_code_55': 'sum',
        'weather_code_71': 'sum',
        'weather_code_75': 'sum',
        'weather_code_73': 'sum',
    }
).reset_index()


quarterly_summary

C:\Users\tinne\AppData\Local\Temp\ipykernel_521360\1472414479.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  weather_data['quarter'] = pd.to_datetime(weather_data['date']).dt.to_period('Q')


latitude   longitude quarter temperature_2m_mean               \
                                                   mean          sum   
0     20.902977 -156.207483  2015Q1           23.107811  2079.703006   
1     20.902977 -156.207483  2015Q2           24.647683  2242.939170   
2     20.902977 -156.207483  2015Q3           27.289700  2510.652412   
3     20.902977 -156.207483  2015Q4           25.447535  2341.173230   
4     20.902977 -156.207483  2016Q1           23.835366  2169.018340   
...         ...         ...     ...                 ...          ...   
1645  61.370716 -152.404419  2022Q1          -16.669162 -1500.224616   
1646  61.370716 -152.404419  2022Q2           -8.186693  -744.989041   
1647  61.370716 -152.404419  2022Q3           -4.537017  -417.405525   
1648  61.370716 -152.404419  2022Q4          -15.110500 -1390.165955   
1649  61.370716 -152.404419  2023Q1          -15.245667   -15.245667   

     relative_humidity_2m_mean              wind_speed_10m_max  \
                          mean          sum                max   
0                    73.154187  6583.876815          41.500053   
1                    73.468107  6685.597706          44.670470   
2                    76.519537  7039.797368          40.104060   
3                    76.703521  7056.723919          46.249844   
4                    70.748610  6438.123503          38.706280   
...                        ...          ...                ...   
1645                 73.232969  6590.967244          15.175612   
1646                 69.424942  6317.669709           9.021574   
1647                 89.136554  8200.562925          10.137692   
1648                 64.777468  5959.527062          12.224107   
1649                 87.541890    87.541890           8.891343   

     temperature_2m_max relative_humidity_2m_max  ... weather_code_65  \
                    max                      max  ...             sum   
0             29.044500                98.460330  ...               0   
1             30.744501                94.885765  ...               0   
2             31.844501                96.768500  ...               0   
3             30.494501                95.030210  ...               0   
4             29.694500                94.330864  ...               0   
...                 ...                      ...  ...             ...   
1645         -10.366501                98.426600  ...               0   
1646           3.783500                99.622440  ...               0   
1647           3.733500               100.000000  ...               0   
1648          -1.316500               100.000000  ...               0   
1649         -13.816501                93.157390  ...               0   

     weather_code_3 weather_code_2 weather_code_0 weather_code_51  \
                sum            sum            sum             sum   
0                18             10              1              28   
1                 7              5              0              48   
2                 5              2              0              53   
3                 3             10              0              39   
4                12             15              1              29   
...             ...            ...            ...             ...   
1645             13              0              6               0   
1646             43              5              3               1   
1647             16              0              0               9   
1648             22              5              5               0   
1649              0              0              0               0   

     weather_code_1 weather_code_55 weather_code_71 weather_code_75  \
                sum             sum             sum             sum   
0                 9               7               0               0   
1                 7               3               0               0   
2                 5               4               0               0   
3       

In [10]:
# change multiindex columns to single index
quarterly_summary.columns = ['_'.join(col).strip() for col in quarterly_summary.columns.values]
quarterly_summary.columns = quarterly_summary.columns.str.replace(' ', '')
quarterly_summary.columns = quarterly_summary.columns.str.replace(')', '')
quarterly_summary.columns = quarterly_summary.columns.str.replace('mean_', 'mean')
quarterly_summary.columns = quarterly_summary.columns.str.replace('sum_', 'sum')
quarterly_summary.columns = quarterly_summary.columns.str.replace('max_', 'max')
quarterly_summary.columns = quarterly_summary.columns.str.replace('min_', 'min')

In [11]:
quarterly_summary.columns

Index(['latitude_', 'longitude_', 'quarter_', 'temperature_2m_meanmean',
       'temperature_2m_meansum', 'relative_humidity_2m_meanmean',
       'relative_humidity_2m_meansum', 'wind_speed_10m_maxmax',
       'temperature_2m_maxmax', 'relative_humidity_2m_maxmax',
       'temperature_2m_minmin', 'relative_humidity_2m_minmin',
       'precipitation_hours_sum', 'weather_code_53_sum', 'weather_code_63_sum',
       'weather_code_61_sum', 'weather_code_65_sum', 'weather_code_3_sum',
       'weather_code_2_sum', 'weather_code_0_sum', 'weather_code_51_sum',
       'weather_code_1_sum', 'weather_code_55_sum', 'weather_code_71_sum',
       'weather_code_75_sum', 'weather_code_73_sum'],
      dtype='object')

In [12]:
# change columns names
quarterly_summary = quarterly_summary.rename(columns=
               {"latitude_": 'latitude', 
                "longitude_": 'longitude',
                "quarter_": 'quarter',
                'temperature_2m_meanmean': 'temperature_2m_mean',
                'temperature_2m_meansum': 'temperature_2m_sum',
                'relative_humidity_2m_meanmean': 'relative_humidity_2m_mean', 
                'relative_humidity_2m_meansum': 'relative_humidity_2m_sum',
                'wind_speed_10m_maxmax': 'wind_speed_10m_max',
                'temperature_2m_maxmax': 'temperature_2m_max',
                'relative_humidity_2m_maxmax': 'relative_humidity_2m_max',
                'temperature_2m_minmin': 'temperature_2m_min',
                'relative_humidity_2m_minmin': 'relative_humidity_2m_min',
                "weather_code_53_sum": 'moderate_drizzle_sum',
                "weather_code_63_sum": 'moderate_rain_sum',
                "weather_code_61_sum": 'light_rain_sum',
                "weather_code_65_sum": 'heavy_rain_sum',
                "weather_code_3_sum": 'overcast_sum',
                "weather_code_2_sum": 'partly_cloudy_sum',
                "weather_code_0_sum": 'clear_sky_sum',
                "weather_code_51_sum": 'light_drizzle_sum',
                "weather_code_1_sum": 'mainly_clear_sum',
                "weather_code_55_sum": 'heavy_drizzle_sum',
                "weather_code_71_sum": 'light_snow_sum',
                "weather_code_75_sum": 'heavy_snow_sum',
                "weather_code_73_sum": 'moderate_snow_sum',        
                })

<div style="background-color:#FCE205; padding:10px; border-radius:5px; color:black; font-weight:bold;">
    <h3>Add state names to their respective longitudes and latitudes</h3>
    </div>

In [13]:
# read in us_states_lat_long.csv
ITM_DIR = os.path.join(os.getcwd(), '../data/import')

us_states_lat_long = pd.read_csv(os.path.join(ITM_DIR, 'us_states_lat_long.csv'))

# left join weather data to us_states_lat_long on latitude and longitude
weather_data_quarter_states = pd.merge(us_states_lat_long, quarterly_summary, on=['latitude', 'longitude'], how='left')

weather_data_quarter_states.isnull().sum()

state                        0
latitude                     0
longitude                    0
quarter                      0
temperature_2m_mean          0
temperature_2m_sum           0
relative_humidity_2m_mean    0
relative_humidity_2m_sum     0
wind_speed_10m_max           0
temperature_2m_max           0
relative_humidity_2m_max     0
temperature_2m_min           0
relative_humidity_2m_min     0
precipitation_hours_sum      0
moderate_drizzle_sum         0
moderate_rain_sum            0
light_rain_sum               0
heavy_rain_sum               0
overcast_sum                 0
partly_cloudy_sum            0
clear_sky_sum                0
light_drizzle_sum            0
mainly_clear_sum             0
heavy_drizzle_sum            0
light_snow_sum               0
heavy_snow_sum               0
moderate_snow_sum            0
dtype: int64

In [14]:
# split quarter column into year and quarter for easier merging with bees in notebook 2a
weather_data_quarter_states[['year', 'quarter']] = weather_data_quarter_states['quarter'].astype(str).str.split('Q', expand=True)

In [15]:
weather_data_quarter_states

,state,latitude,longitude,quarter,temperature_2m_mean,temperature_2m_sum,relative_humidity_2m_mean,relative_humidity_2m_sum,wind_speed_10m_max,temperature_2m_max,...,overcast_sum,partly_cloudy_sum,clear_sky_sum,light_drizzle_sum,mainly_clear_sum,heavy_drizzle_sum,light_snow_sum,heavy_snow_sum,moderate_snow_sum,year
0,Alabama,32.806671,-86.79113,1,9.138389,822.454992,72.118324,6490.649186,31.782108,25.149500,...,36,5,4,9,5,2,1,0,0,2015
1,Alabama,32.806671,-86.79113,2,22.649935,2061.144045,73.748212,6711.087248,22.737070,35.149500,...,28,7,2,9,2,7,0,0,0,2015
2,Alabama,32.806671,-86.79113,3,26.056180,2397.168552,73.825042,6791.903907,18.014393,36.249500,...,25,6,1,20,5,8,0,0,0,2015
3,Alabama,32.806671,-86.79113,4,16.406248,1509.374798,75.721304,6966.360008,30.955812,29.049500,...,38,4,7,10,4,2,0,0,0,2015
4,Alabama,32.806671,-86.79113,1,10.685764,972.404481,70.392922,6405.755898,32.777794,28.049500,...,29,6,13,10,5,3,0,0,0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,Wyoming,42.755966,-107.30249,1,-8.064703,-725.823311,68.191139,6137.202541,45.143770,14.501500,...,56,0,1,1,0,0,14,3,15,2022
1646,Wyoming,42.755966,-107.30249,2,6.131697,557.984432,54.287062,4940.122644,57.718000,29.551500,...,28,5,3,19,2,1,6,6,13,2022
1647,Wyoming,42.755966,-107.30249,3,18.094526,1664.696387,38.602263,3551.408216,40.732533,31.551502,...,28,6,8,23,12,2,0,0,0,2022
1648,Wyoming,42.755966,-107.30249,4,-4.583916,-421.720318,66.597217,6126.943967,56.993810,18.251501,...,43,1,9,3,3,0,11,3,19,2022


In [16]:
# Save the quarterly summary to a CSV file
OUT_DIR = os.path.join(os.getcwd(), '../data/intermediate')

weather_data_quarter_states.to_csv(os.path.join(OUT_DIR, 'quarterly_weather_summary.csv'), index=False)